In [16]:
import torch
import torch.nn.functional as F
from torch import nn
import math
from transformers.models.distilbert.modeling_distilbert import MultiHeadSelfAttention
from transformers.configuration_utils import PretrainedConfig
from transformers import DistilBertForSequenceClassification, DistilBertConfig



In [17]:
student_config = DistilBertConfig(output_hidden_states = True)
student_config.distance_metric = 'manhattan'

In [18]:
print(student_config)

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "distance_metric": "manhattan",
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.41.2",
  "vocab_size": 30522
}

